# Introduction to CUDA C++ - IDSLab Seminar [11/03/2022]

A simple introduction to CUDA C++. Knowledge of C++ and memory allocation is recommended as CUDA C++ uses pointers, variable addresses and memory allocation. 

In [1]:
from IPython.display import IFrame
IFrame("slides.pdf", width=800, height=400)

## The Basics

In CUDA C++ we program the CPU code and GPU code (Kernel) together
```cpp
#include <stdio.h> //C++ standard library for input and output, we need printf

__global__ void GPUKernel(){//Notice __global__? We are declaring this as a kernel

  printf("Hi, this is GPU.\n");
}

int main(){ //This will run on CPU

  printf("Hello, this is CPU.\n");

  GPUKernel<<<1, 1>>>(); //Execute kernel on GPU
  cudaDeviceSynchronize(); //Synchronize GPU threads
}
```

### Compiling CUDA C++

We use `nvcc` to compile CUDA C++. `nvcc` is included with CUDA ToolKit so every GPU server has `nvcc`.

CUDA C++ files end with `.cu` extenstion.

In [2]:
!nvcc -o hello hello.cu -run

Hello, this is CPU.
Hi, this is GPU.


### Let's break it down!
#### GPU Code (Kernel)
```cpp
__global__ void GPUKernel(){//Notice __global__? We are declaring this as a kernel

  printf("Hi, this is GPU.\n");
}
```
`__global__` declares the function is a GPU kernel. But we must call the kernel from the CPU to execute it on GPU.

Warning!: Never print in a kernel. This is just an example to show how kernels work. Printing in kernel will greatly hinder the GPU performance and can lead to many bugs.

#### CPU Code
```cpp
int main(){ //This will run on CPU

  printf("Hello, this is CPU.\n");

  GPUKernel<<<1, 1>>>(); //Execute kernel on GPU
  cudaDeviceSynchronize(); //Synchronize GPU threads
}
```
We use `functionName<<<1, 1>>>()` to execute the kernel. `1, 1` is blocks and threads respectively, we can leave as 1 for  now. CUDA will automatically deploy the kernel to each SM.

We must call `cudaDeviceSynchronize()` before starting a new task. The  CPU and GPU run simultaneously, so we must manually tell the CPU to wait for all GPU treads to finish execution.

(Advanced Tip: Threads and blocks can be multidimensional and must be set appropriately for the task and data for optimal performance.)

(Advanced Tip: If we are running asynchronous tasks, we don't need to call `cudaDeviceSynchronize()`.)

## Confused? Let's try a "small" numerical example - addition of two 1D arrays

Firstly, let's consider how we would add two 1D arrays together using the only the CPU.

### CPU Implementation

Let's have three arrays  `a`, `b`, and `c`, of equal length, `N`. Where `a` and `b` are the two arrays we want to sum and `c` is the result array.

```cpp
for(int i = 0; i < N; i++){
    c[i] = a[i] + b[i]
}

```
Simple right? But each element of the arrays is summed one by one, so this process is serial.

What if we could add all the elements simultaneously? We can, using the parallel threads on a GPU! 


(Let's ignore CPU multithreading, as the available threads on a CPU [AMD Threadripper PRO 3995WX = 64 cores/128 threads] compared to a GPU [NVIDIA A100 = 6912 cores/threads] is orders of magnitude smaller.)

### CUDA Implementation

```cpp
#include <stdio.h> //C++ standard library for input and ouput, we need printf

#define N 100 //C++ define tells the compiler to replace all N to 100 before compiling

__global__ void addArrays(int *a, int *b, int *c){ //Notice __global__? We are declaring this as a kernel

  int idx = threadIdx.x; //threadIdx allows the kernel to identify what thread it is in
  c[idx] = a[idx] + b[idx];
}

void initArray(int *array, int num){ //Helper function - set every element of the array to a value

  for(int i = 0; i < N; ++i){
    array[i] = num;
  }
}

int main(){
  int *a; //Create a pointer for an int for first array
  int *b; //Create a pointer for an int for second array
  int *c; //Create a pointer for an int for result array

  size_t size = N * sizeof(int); //Calculate the memory size of an int array with length N

  //Allocating memory
  //cudaMallocManaged will allocate memory on both the CPU DRAM and GPU memory
  //cudaMallocManaged needs the memory address (&variable) and size to be allocated
  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  //Initialize our arrays, let's just set every value in the array to one value for now
  initArray(a, 10);
  initArray(b, 32);
  initArray(c, 0);

  //Execute our kernel with our arrays that we have initialized
  addArrays<<<1, N>>>(a, b, c);
  cudaDeviceSynchronize(); //Wait for all threads to be executed

  //Let's check every value in the array has summed correctly
  for(int i = 0; i < N; i++){
    if(c[i] != 42){
      printf("FAIL: array[%d] - %d does not equal %d\n", i, c[i], 42);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");

  //Time to release the memory
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}
```

In [3]:
!nvcc -o vector_add vector_add.cu -run

Success! All values calculated correctly.


### Lets break it down!
#### GPU Code (Kernel)

```cpp
__global__ void addArrays(int *a, int *b, int *c){ //Notice __global__? We are declaring this as a kernel

  int idx = threadIdx.x; //threadIdx allows the kernel to identify what thread it is in
  c[idx] = a[idx] + b[idx];
}
```

You many of noticed our kernel always returns `void`. The kernel never returns any variables, it works directly on the variables given via pointers. This is why we must allocate the memory for the results array before executing the kernel.

`threadIdx` is an object provided CUDA C++ and it is multidemnsion up to three dimensions (`x`, `y` and `z`). `threadIdx` allows the kernel to identifiy which thread index it is running on. As we are doing a single 1D problem we only need `threadIdx.x`. 

If our array has a length of 100 we can set the number of threads to 100 so there is one thread per element. Then we can simply use the `threadIdx` to sum each element together. (We dont need the for loop like the CPU implemnetation.)


#### CPU Code
```cpp
int main(){
  int *a; //Create a int pointer for the first array
  int *b; //Create a int pointer for the second array
  int *c; //Create a int pointer for the result array

  size_t size = N * sizeof(int); //Calculate the memory size of an int array with length N

  //Allocating memory
  //cudaMallocManaged will allocate memory on both the CPU DRAM and GPU memory
  //cudaMallocManaged needs the memory address (&variable) and size to be allocated
  cudaMallocManaged(&a, size);
  cudaMallocManaged(&b, size);
  cudaMallocManaged(&c, size);

  //Initialize our arrays, let's just set every value in the array to one value for now
  initArray(a, 10);
  initArray(b, 32);
  initArray(c, 0);

  //Execute our kernel with our arrays that we have initialized
  addArrays<<<1, N>>>(a, b, c);
  cudaDeviceSynchronize(); //Wait for all threads to be executed

  //Let's check every value in the array has summed correctly
  for(int i = 0; i < N; i++){
    if(c[i] != 42){
      printf("FAIL: array[%d] - %d does not equal %d\n", i, c[i], 42);
      exit(1);
    }
  }
  printf("Success! All values calculated correctly.\n");

  //Time to release the memory
  cudaFree(a);
  cudaFree(b);
  cudaFree(c);
}
```

This time the CPU code is more complex because we need to manage memory allocation. `cudaMallocManaged()` allows us to allocate CPU and GPU memory and automatically sync between the two. If we update a variable in the GPU kernel, it is also updated in the CPU memory (unified memory). This allows us to easily perform task on data. If we only want to allocate on GPU memory, we would use `cudaMalloc()`.

Before using `cudaMallocManaged()` we must declare a pointer to the variable type we want to use. Here we use `int *a;` as we are working with integers. We also need to know the size the array will consume in the memory `size_t size = N * sizeof(int);` we are creating an array of integers with length N. Now we have the pointers for each array and the size we can allocate the memory using `cudaMallocManaged(&a, size)`. `&` before a variable in C++ returns the memory address of the variable. 

Now we can initialize the arrays with some integers using `initArray()` helper function.

Let's execute our `addArrays()` kernel using `addArrays<<<1, N>>>(a, b, c)`. Here we have set `<<<1, N>>>` this will deploy our kernel with N threads, one thread for each element in the arrays.

Next `cudaDeviceSynchronize()`, to ensure all threads have completed.

Let's check the result is correct on the CPU by checking every element is equal to 42 (10+32).

One last step before we finish, we must release the memory allocation using `cudaFree()`.

## Review
This is a very basic introduction just to highlight the functionality of CUDA C++. We have learned how to implement GPU kernels and learned the basics of managing GPU memory.

CUDA C++ can be powerful when using more advanced features like shared memory.

### CUDA C++ Cheat Sheet

`__global__` declares the function is a GPU kernel.

`functionName<<<1, 1>>>()` execute the kernel, with `1, 1` blocks and threads.

`cudaDeviceSynchronize()` synchronize all the threads.

`cudaMallocManaged(&a, size)` allocate memory in CPU and GPU.

`cudaFree()` release memory.